# LW4e as a Difference of Fluxes

In [1]:
#For the LaTeX equations (such as eqnarray) in this document to work, include the following in file
#~/.jupyter/_config.yml
#
#parse:
#  myst_enable_extensions:  # default extensions to enable in the myst parser. See https://myst-parser.readthedocs.io/en/latest/using/syntax-optional.html
#     - amsmath
#
#(the default ~/.jupyter/_config.yml will have amsmath commented out)
import numpy as np
from numpy import exp
import sympy as sy
from sympy import latex
from sympy import I   # symbol for sqrt of -1
from sympy.matrices import Matrix, MatrixSymbol
from fractions import Fraction as Fr
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

## Derivation of LW4e
Using spatial gradients from a quartic polynomial. If
\begin{eqnarray}
\psi &=& ax^4 + bx^3 + cx^2 + dx + e
\end{eqnarray}
then
\begin{eqnarray}
\frac{\partial\psi}{\partial x} &=& 4ax^3 + 3bx^2 + 2cx + d \\
\frac{\partial^2\psi}{\partial x^2} &=& 12ax^2 + 6bx + 2c \\
\frac{\partial^3\psi}{\partial x^3} &=& 24ax + 6b \\
\frac{\partial^4\psi}{\partial x^4} &=& 24a
\end{eqnarray}
Assuming $\Delta x=1$, we know
\begin{eqnarray}
\psi_{i-2} = \psi_0 &=& 2^4 a - 2^3 b + 2^2 c - 2 d + e \\
\psi_{i-1} = \psi_1 &=& a - b + c - d + e \\
\psi_{i} = \psi_2 &=& e \\
\psi_{i+1} = \psi_3 &=& a+b+c+d+e \\
\psi_{i+2} = \psi_4 &=& 2^4 a + 2^3 b + 2^2 c + 2 d + e \\
\end{eqnarray}
So $(a,b,c,d,e)$ is the solution of the matrix equation
\begin{equation*}
\left( \begin{array}{ccccc}
2^4 & -2^3 & 2^2 & -2 & 1 \\
1 & -1 & 1 & -1 & 1 \\
0 & 0 & 0 & 0 & 1 \\
1 & 1 & 1 & 1 & 1 \\
2^4 & 2^3 & 2^2 & 2 & 1 \\
\end{array} \right) 
\left( \begin{array}{c}
a \\ b \\ c \\ d \\ e
\end{array} \right) 
=
\left( \begin{array}{c}
\psi_0 \\ \psi_1 \\ \psi_2 \\ \psi_3 \\ \psi_4
\end{array} \right) 
\end{equation*}

In [2]:
# The sympy symbols
PolyCoeffs = sy.Matrix(sy.symarray("PolyCoeffs", (5,))) # Coefficients a,b,c,d,e of the polynomial
Psi = sy.Matrix(sy.symarray("Ψ", (5,)), real=True)    # Grid point values at i-2, i-1, i, i+1 and i+2
polyM = Matrix([[2**4,-2**3,2**2,-2,1], [1,-1,1,-1,1], [0,0,0,0,1], [1,1,1,1,1],
                [2**4,2**3,2**2,2,1]])
PolyCoeffs = polyM.solve(Psi)
Ddx = PolyCoeffs[3]
D2dx2 = 2*PolyCoeffs[2]
D3dx3 = 6*PolyCoeffs[1]
D4dx4 = 24*PolyCoeffs[0]
# From the gradients we get the explicit LW4e  increment based on the Courant number
C = sy.symbols("C", real=True)
LW4e = - C*Ddx + C**2/2*D2dx2 - C**3/6*D3dx3 + C**4/24*D4dx4
print('LW4e increment is', LW4e)
print('LW4e increment as coefficients of grid points is')
LW4e = sy.collect(sy.expand(LW4e), Psi)
LW4e
#print(latex(LW4e))

LW4e increment is C**4*(Ψ_0 - 4*Ψ_1 + 6*Ψ_2 - 4*Ψ_3 + Ψ_4)/24 - C**3*(-Ψ_0/2 + Ψ_1 - Ψ_3 + Ψ_4/2)/6 + C**2*(-Ψ_0/12 + 4*Ψ_1/3 - 5*Ψ_2/2 + 4*Ψ_3/3 - Ψ_4/12)/2 - C*(Ψ_0/12 - 2*Ψ_1/3 + 2*Ψ_3/3 - Ψ_4/12)
LW4e increment as coefficients of grid points is


Ψ_0*(C**4/24 + C**3/12 - C**2/24 - C/12) + Ψ_1*(-C**4/6 - C**3/6 + 2*C**2/3 + 2*C/3) + Ψ_2*(C**4/4 - 5*C**2/4) + Ψ_3*(-C**4/6 + C**3/6 + 2*C**2/3 - 2*C/3) + Ψ_4*(C**4/24 - C**3/12 - C**2/24 + C/12)

In [4]:
# Find Ψ_l and Ψ_r so that LW4e = -Ψ_r + Ψ_l
LW4eR = LW4e # The residual of LW4e after removal of -Ψ_r + Ψ_l
[Psil, Psir] = sy.symbols("Ψ_l, Ψ_r", real=True)
Psil = Psir = sy.S.Zero
for j in range(len(Psi)-1):
    Psil = Psil + Psi[j]*LW4eR.coeff(Psi[j])
    Psir = Psir + Psi[j+1]*LW4eR.coeff(Psi[j])
    LW4eR = sy.collect(sy.expand(LW4e + Psir - Psil), Psi)

print('Residual =', LW4eR, '\nThe right flux is')
Psir

Residual = 0 
The right flux is


Ψ_1*(C**4/24 + C**3/12 - C**2/24 - C/12) + Ψ_2*(-C**4/8 - C**3/12 + 5*C**2/8 + 7*C/12) + Ψ_3*(C**4/8 - C**3/12 - 5*C**2/8 + 7*C/12) + Ψ_4*(-C**4/24 + C**3/12 + C**2/24 - C/12)